In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import sys
import os
import pandas as pd
import plotly.express as px
from sklearn.preprocessing import StandardScaler
import ast

sys.path.append(os.path.abspath('../src/'))
from ActivationFunctions import *
from NeuralNetwork import *
from MyUtils import *
from ModelSelection import *

%matplotlib inline

In [ ]:
tr_norm_df = pd.read_csv('../data/divided_std_train_0_8.csv')
test_norm_df = pd.read_csv('../data/divided_std_test_0_2.csv')

tr_df = pd.read_csv('../data/divided_train_0_8.csv')
test_df = pd.read_csv('../data/divided_test_0_2.csv')

ML_cup_train = pd.read_csv('../data/ML-CUP23-TR.csv', header=None, index_col=0, comment='#').sample(frac=1, random_state=RANDOM_STATE)
TR_INPUT = 10
TR_OUTPUT = 3
scaler_in = StandardScaler()
scaler_out = StandardScaler()
ML_cup_train_norm = ML_cup_train.values.copy()

ML_cup_train_norm[:,:TR_INPUT] = scaler_in.fit_transform(ML_cup_train_norm[:,:TR_INPUT])
ML_cup_train_norm[:,TR_INPUT:] = scaler_out.fit_transform(ML_cup_train_norm[:,TR_INPUT:])

training_len = len(ML_cup_train_norm)

In [ ]:
def load_obj(path):    
    with open(path, 'r') as file:
        ret = json.load(file)
    for el in ret:
        with open(el['nn_file_name'], 'r') as file:
                el['model'] = NeuralNetwork.fromJSON(file.read())
    return ret
            
def save_obj(obj, path):
    for i in obj:
        with open(i['nn_file_name'], 'w+') as file:
            file.write(i['model'].toJSON())
        i['model'] = None
    json.dump(obj, path, default=str)

In [ ]:
hidden_len = 32
hidden_fun = 'sigmoid'
output_fun = 'identity'
sigmoid_l1 = create_stratified_topology([TR_INPUT,hidden_len,TR_OUTPUT], 
                                      [[None,[]]]*TR_INPUT + [[hidden_fun, [1]]]*hidden_len + [[output_fun, []]]*TR_OUTPUT)
NeuralNetwork.display_topology(sigmoid_l1)

In [ ]:
def load_obj(path):    
    with open(path, 'r') as file:
        ret = json.load(file)
    for el in ret:
        with open(el['nn_file_name'], 'r') as file:
                el['model'] = NeuralNetwork.fromJSON(file.read())
    return ret
            
def save_obj(obj, path):
    for i in obj:
        with open(i['nn_file_name'], 'w+') as file:
            file.write(i['model'].toJSON())
        i['model'] = None
    json.dump(obj, path, default=str)

In [ ]:
def remove_useless_val(x):
    a =['learning_rate', 'lr_decay_tau', 'alpha_momentum']
    b =['adamax_learning_rate', 'exp_decay_rate_1', 'exp_decay_rate_2']
    if x['adamax']:
        for i in a:
            if i in x.keys():
                x[i] = None
    else:
        for i in b:
            if i in x.keys():
                x[i] = None   
    return x

In [ ]:
columns_order = ['topology', 'stats',
 'batch_size',
 'min_epochs',
 'max_epochs',
 'patience',
 'error_increase_tolerance',
 'lambda_tikhonov',
 
 'learning_rate',
 'alpha_momentum',
 'lr_decay_tau',
 
 'adamax',
 'adamax_learning_rate',
 'exp_decay_rate_1',
 'exp_decay_rate_2',
 
 'mean_mean_euclidean_error',
 'mean_mean_squared_error',
 'var_mean_euclidean_error',
 'var_mean_squared_error',
 'mean_best_validation_training_error']

In [ ]:
results_files = ['bagging_sub_model.csv']
topologies = ['32_sigmoid']
folder = '../data/gs_data/'
topologies_dict = {}
gs_results = []
for i, f in enumerate(results_files):
    if os.path.isfile(folder+ f):
        dummy = pd.read_csv(folder + f)
        topologies_dict[topologies[i]] = ast.literal_eval(dummy['topology'][0])
        dummy['topology'] = topologies[i]
        
        gs_results.append(dummy)
    


orig_df = pd.concat(gs_results)

In [ ]:
columns_order = [x for x in columns_order if x in orig_df.columns]
orig_df = orig_df[columns_order]

order_by = 'mean_mean_euclidean_error'
orig_df.sort_values(by=[order_by], inplace=True)
orig_df = orig_df.reset_index(drop=True)
gs_results = orig_df.drop(['stats'], axis=1)

In [ ]:
gs_results = gs_results.apply(remove_useless_val, axis=1)
for i in gs_results.columns[1:]:
    gs_results[i] = gs_results[i].astype(float)

In [ ]:
numerical_col = list(gs_results.columns)
numerical_col.remove('topology')
st_opt_col = ['learning_rate','lr_decay_tau','alpha_momentum']
adamax_opt_col = ['adamax','adamax_learning_rate','exp_decay_rate_1','exp_decay_rate_2']
metrics_col = [x for x in gs_results.columns if x.startswith(('var', 'mean'))]
general_col = [item for item in list(gs_results.columns) if item not in st_opt_col and item not in metrics_col and item not in adamax_opt_col]
tr_input_col = [item for item in list(gs_results.columns) if item not in metrics_col]

In [ ]:
n_models = 33
tr_len = 533
max_epochs = 500
mod = []

met = [ErrorFunctions.mean_squared_error, ErrorFunctions.mean_euclidean_error, ]

In [ ]:
if False:
    interesting_model = load_obj('../data/net/models_7_final_retr/models.json')
    interesting_model_2 = load_obj('../data/net/a1/models.json')
    interesting_model_3 = load_obj('../data/net/a2/models.json')
    interesting_model_4 = load_obj('../data/net/a3/models.json')
    interesting_model_5 = load_obj('../data/net/a4/models.json')

    scarto = 4
    for i, el in enumerate(interesting_model_2):
        el['nn_file_name'] = '../data/net/models_7_final_retr/model_' + str(i + scarto) + '.json'
        el['index'] = i + scarto
    interesting_model += interesting_model_2
    scarto = 4 + 11
    for i, el in enumerate(interesting_model_3):
        el['nn_file_name'] = '../data/net/models_7_final_retr/model_' + str(i + scarto) + '.json'
        el['index'] = i + scarto
    interesting_model += interesting_model_3
    scarto = 4 + 11 + 11
    for i, el in enumerate(interesting_model_4):
        el['nn_file_name'] = '../data/net/models_7_final_retr/model_' + str(i + scarto) + '.json'
        el['index'] = i + scarto
    interesting_model += interesting_model_4
    scarto = 4 + 11 + 11 + 3
    for i, el in enumerate(interesting_model_5):
        el['nn_file_name'] = '../data/net/models_7_final_retr/model_' + str(i + scarto) + '.json'
        el['index'] = i + scarto
    interesting_model += interesting_model_5

    with open('../data/net/models_7_final_retr/ens_curves_tr.json', 'r') as file:
        predictions_accumul_tr = np.array(json.load(file))

    with open('../data/net/a1/ens_curves_tr.json', 'r') as file:
        predictions_accumul_tr += np.array(json.load(file))
        
    with open('../data/net/a2/ens_curves_tr.json', 'r') as file:
        predictions_accumul_tr += np.array(json.load(file))
        
    with open('../data/net/a3/ens_curves_tr.json', 'r') as file:
        predictions_accumul_tr += np.array(json.load(file))
        
    with open('../data/net/a4/ens_curves_tr.json', 'r') as file:
        predictions_accumul_tr += np.array(json.load(file))
        
    with open('../data/net/models_7_final_retr/models.json', 'w+') as file:
        save_obj(interesting_model, file)
        
    with open('../data/net/models_7_final_retr/ens_curves_tr.json', 'w+') as file:
        file.write(json.dumps(predictions_accumul_tr.tolist()))

In [ ]:
interesting_model = load_obj('../data/net/models_7_final_retr/models.json')
with open('../data/net/models_7_final_retr/ens_curves_tr.json', 'r') as file:
    predictions_accumul_tr = np.array(json.load(file))/n_models

In [ ]:
def ensemlbe(compl_obj, set):

    output = []#np.empty((len(set), TR_OUTPUT))
    
    for el in set:
        dummy = np.zeros(TR_OUTPUT)
        for j in range(len(compl_obj)):
            dummy += compl_obj[j]['model'].predict(el)

        output.append(dummy/len(compl_obj))
        #print(dummy/len(compl_obj))
        
    return output
    

In [ ]:
str_prediction_tr = ensemlbe(interesting_model, ML_cup_train_norm[:,:TR_INPUT])

prediction_tr = scaler_out.inverse_transform(str_prediction_tr)

std_error_tr = ErrorFunctions.mean_euclidean_error(str_prediction_tr, ML_cup_train_norm[:,TR_INPUT:])

error_tr = ErrorFunctions.mean_euclidean_error(prediction_tr, ML_cup_train.values[:,TR_INPUT:])

In [ ]:
print('std_error_tr:', std_error_tr)
print('error_tr:', error_tr)

In [ ]:
ens_err_train = [[], []]

for i in range(len(met)):
    for epoch in range(max_epochs):
        ens_err_train[i].append(met[i](predictions_accumul_tr[epoch], ML_cup_train_norm[:,TR_INPUT:]))

In [ ]:
fig, ax1 = plt.subplots(1, figsize=(5,4))
fig.suptitle('MEE Learning Curves Ensembling And Sub-models')

for dummy in interesting_model:
    stats = dummy['stats']
        
    ax1.plot(list(range(len(stats['training_mean_euclidean_error']))), stats['training_mean_euclidean_error'], color = 'C0', alpha=0.05)
   
    
ax1.plot(list(range(len(ens_err_train[1]))), ens_err_train[1], color = 'C0', label='training_error')

ax1.legend();

In [ ]:
ML_cup_test = pd.read_csv('../data/ML-CUP23-TS.csv', header=None, index_col=0, comment='#')
ML_cup_train = pd.read_csv('../data/ML-CUP23-TR.csv', header=None, index_col=0, comment='#')

in_scaler = StandardScaler()
in_scaler.fit(ML_cup_train.values[:,:TR_INPUT])

ML_cup_test_std = in_scaler.transform(ML_cup_test)
ML_cup_test_std.shape

In [ ]:
ML_cup_pred_std = ensemlbe(interesting_model, ML_cup_test_std)

In [ ]:
ML_cup_pred_df = pd.DataFrame(scaler_out.inverse_transform(ML_cup_pred_std))
ML_cup_pred_df

In [ ]:
ML_cup_pred_df.index += 1
ML_cup_pred_df.to_csv('../data/AIdra_ML-CUP23-TS.csv', header=False)